In [ ]:
#libraries
import pandas as pd
import datetime as dt
import plotly.express as px
import plotly.offline as pyo
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns


In [ ]:
##Reading datasets
cab_data = pd.read_csv(r"C:\Users\diile\Desktop\Cab_Data.csv") 
city_data = pd.read_csv(r"C:\Users\diile\Desktop\City.csv") 
customer_data = pd.read_csv(r"C:\Users\diile\Desktop\Customer_ID.csv") 
transaction_data = pd.read_csv(r"C:\Users\diile\Desktop\Transaction_ID.csv") 

In [ ]:
city_data

In [ ]:
customer_data.isnull().sum()


In [ ]:
city_data.isnull().sum()


In [ ]:
transaction_data.isnull().sum()


In [ ]:
customer_data.isnull().sum()


In [ ]:
cab_data.info()
city_data.info()
customer_data.info()
transaction_data.info()

In [ ]:
#handling column names
for col in cab_data.columns:
    if ' ' in col:
        cab_data = cab_data.rename(columns={col:col.replace(' ','_')})

In [ ]:
#handling column names
for col in transaction_data.columns:
    if ' ' in col:
        transaction_data = transaction_data.rename(columns={col:col.replace(' ','_')})

In [ ]:
for col in customer_data.columns:
    if ' ' in col:
        customer_data = customer_data.rename(columns={col:col.replace(' ','_')})

In [ ]:

cab_data['Date_of_Travel'] = pd.TimedeltaIndex(cab_data['Date_of_Travel'], unit='d') + dt.datetime(1899, 12, 30)


In [ ]:
cab_data['Date_of_Travel']

In [ ]:
months=[]
years =[]
for i in range(len(cab_data['Date_of_Travel'])):
    months.append(cab_data['Date_of_Travel'][i].month)
    years.append(cab_data['Date_of_Travel'][i].year)
cab_data['Month'] = months
cab_data['Year'] = years

In [ ]:
cab_data = cab_data.drop(['Date_of_Travel'],axis=1)


In [ ]:
cab_data.head()

In [ ]:
#Calculating profits for each transaction.
cab_data['profit'] = cab_data['Price_Charged'] - cab_data['Cost_of_Trip']
cab_data.head()


In [ ]:
#general view of transaction costs grouped by company

fig = px.histogram(cab_data, x="Company", y="profit",color="Company",
                   title='Profit Grouped by Companies ')
fig.show()

#Sum of profit of yellow cab is way more than pink cab. Close to 39M.

In [ ]:
#Yearly profit grouped by companies 
fig = px.histogram(cab_data, x="Year", y="profit",color="Company",
                   title='Profit Grouped by Companies per Year ', barmode="group",
                  category_orders={"Year": ["2016", "2017", "2018"],
                              "Company": ["Pink Cab", "Yellow Cab"]})
fig.show()
#Yellow cab's profit is always higher than pink. Both pink and yellow had decrease in their profits in 2017.
#Yellow cab lost nearly 3M and pink cab lost nearly half a million in 2018.

In [ ]:
plot_tr = cab_data.groupby(['Year','Company']).Transaction_ID.count()
plot_trgen = cab_data.groupby('Company').Transaction_ID.count()
print('Yearly Transaction Analysis',plot_tr)
print('Company Transaction share :',plot_trgen)

In [ ]:
fig = px.histogram(cab_data, x="Month",y="Transaction_ID",color="Company", barmode="group",histfunc="count",
title='Number of Transactions per Month'
)
fig.show()
#yellow cab has more customers every month of the year,especially during month of December which is a holiday month for USA.

In [ ]:
#Creating Master Data 
left = cab_data.set_index(['Transaction_ID'])
right = transaction_data.set_index(['Transaction_ID'])
data = left.join(right)
data

In [ ]:
left = data.set_index([data.index,'Customer_ID'])
right = customer_data.set_index(['Customer_ID'])
data = left.join(right)
data

In [ ]:
left = data.set_index([data.index,'City'])
right = city_data.set_index(['City'])
data = left.join(right)
data

In [ ]:
data = data.reset_index()


In [ ]:
data.duplicated()

In [ ]:
data

In [ ]:
data = data.rename(columns={'Income_(USD/Month)':'Income'})
YellowCab = data[data['Company']=='Yellow Cab']
PinkCab = data[data['Company']=='Pink Cab']


In [ ]:
city_n = data.groupby('City').Transaction_ID.count()
city_n

In [ ]:
#About a quarter of all transactions are in New York.
lab = city_n.index
val = city_n.values
trace = go.Pie(labels=lab, 
                values=val, 
                # Seting values to 
              )
datapie = [trace]
layout = go.Layout(title="Transaction Distribution")
fig = go.Figure(data = datapie,layout = layout)
fig.show()
#About a quarter of all transactions are in New York.

In [ ]:
mtNYC = data[data['City']=='NEW YORK NY']
mtNYC_ =data.groupby('Company').Transaction_ID.count()

In [ ]:
#Which company has more transactions in NYC?
lab = mtNYC_.index
val = mtNYC_.values
trace = go.Pie(labels=lab, 
                values=val, 
                # Seting values to 
              )
datapie = [trace]
layout = go.Layout(title="Transaction Distribution in NYC per companies")
fig = go.Figure(data = datapie,layout = layout)
fig.show()
#Yellow cab has %76.4 of users.

In [ ]:
#Are there any cities which pink cab has more transactions?
fig = px.histogram(data, x="City",y="Transaction_ID",color="Company", barmode="group",histfunc="count",
title='Number of Transactions per Month'
)
fig.show()
#In Nashville,Sacramento and San Diego CA there are more transactions.

In [ ]:
#There should a positive correlation between km travelled and price charged.
PinkCab['KM_Travelled'].corr(PinkCab['Price_Charged'])



In [ ]:
YellowCab['KM_Travelled'].corr(YellowCab['Price_Charged'])
#there is a positive correlation for both Pink and Yellow cab.

In [ ]:
#Is it cheaper to use Yellow Cab than Pink Cab in NYC ?
#First,lets look at Yellow Cab's prices.
plt.figure(figsize=(10,10))
sns.scatterplot(data=cab_data, x=YellowCab["KM_Travelled"], y=YellowCab['Price_Charged'], hue="City")
plt.ylabel('Yellow Cab Price ')
plt.xlabel('Yellow Cab KM ')

In [ ]:
#After that,lets look at pink cab's.
plt.figure(figsize=(10,10))
sns.scatterplot(data=cab_data, x=PinkCab["KM_Travelled"], y=PinkCab['Price_Charged'], hue="City")
plt.ylabel('Pink Cab Price ')
plt.xlabel('Pink Cab KM ')

In [ ]:
#From the graphs, we can say that Yellow Cab costs more than Pink cab in NYC.
#Also, we can see for Yellow Cab that they charge more than any other city in NYC.

In [ ]:
#Did one of the companies had decrease in customers?
fig = px.histogram(data, x="Company",y="Customer_ID",color="Year", barmode="group",histfunc="count",
title='Customers according to years for each companies'
)
fig.show()
#There is no decrease in customers for each company.

In [ ]:
#Fixing data frame for getting the max user value.
YellowCab=pd.DataFrame(YellowCab)
YellowCab = YellowCab.replace(',','', regex=True)
YellowCab['Users'] = pd.to_numeric(YellowCab.Users)

PinkCab=pd.DataFrame(PinkCab)
PinkCab = PinkCab.replace(',','', regex=True)
PinkCab['Users'] = pd.to_numeric(PinkCab.Users)



In [ ]:
#Both gained customers.Now let's look at which  gained more profit.
#Lets set time frame as 2016-2018 for calculating year-over-year growth in terms of customer number for each company.
#Retrieve Yellow Cab's numbers from 2016 and 2018
#Yellow
#Subtract last year's number from this year's
yellowno=YellowCab.loc[YellowCab['Year'] == 2018, 'profit'].mean() - YellowCab.loc[YellowCab['Year'] == 2016, 'profit'].mean()
#Pink
pinkno=PinkCab.loc[PinkCab['Year'] == 2018, 'profit'].mean() - PinkCab.loc[PinkCab['Year'] == 2016, 'profit'].mean()
#Divide the total by last year's number

yno2=yellowno/YellowCab.loc[YellowCab['Year'] == 2018, 'profit'].mean()
pno2=pinkno/PinkCab.loc[PinkCab['Year'] == 2018, 'profit'].mean()
#Multiply by 100 to get the final percentage
ygrowth=yno2/100
pgrowth=pno2/100
print("Yellow Cab : ", ygrowth,"Pink Cab" ,pgrowth)


In [ ]:
#As shown, yellow cab's growth rate is better. But both is in negative values.
max(ygrowth,pgrowth)

In [ ]:
#Customer share
fig = px.pie(data, values='Customer_ID', names='Company')
fig.show()
#Yellow cab has more share in customers.

In [ ]:
#Is any company preferred more by a certain age range?
fig = px.histogram(data, x="Age",y="Customer_ID",color="Company",histfunc="count",
title='Age Distribution for companies'
)
fig.show()
#As shown, there is no significant difference of choice within age ranges.
#We can say that no matter the age,Yellow cab is preffered more than Pink. 

In [ ]:
#Is any company preferred more by a certain gender?
fig = px.histogram(data, x="Gender",y="Customer_ID",color="Company",histfunc="count",
title='Gender distribution for companies'
)
fig.show()
#From the graph, we can say that yellow cab is preffered more no matter the gender.

In [ ]:
#Since Yellow Cab has more profit and more expensive in some cities, lets check if there is a relationship with social class.
LowerIncome= data[data['Income']<=data['Income'].mean()]
HigherIncome= data[data['Income']>data['Income'].mean()]

In [ ]:
#People with lower income prefers Yellow Cab even though it is more expensive in some cities.(Like NYC)
fig = px.pie(LowerIncome, values='Customer_ID', names='Company')
fig.show()

In [ ]:
##People with higher income prefers Yellow Cab.

fig = px.pie(HigherIncome, values='Customer_ID', names='Company')
fig.show()

We can conclude from the analysis : 
1.Yellow cab is preffered more than pink cab no matter age,income or gender by customers.
2.Yearly profit of yellow cab is always more than pink cab.
3.Growth rate of yellow cab is higher than pink cab.
4.Out of 19 cities, only in three cities there are more transactions done by Pink Cab than Yellow Cab.
5.Yellow cab has more customers every month of the year,especially during month of December which is a holiday month for USA.
6.Yellow cab usually costs more to customers.

After these conclusions,I recommend investing in Yellow Cab.